In [1]:
import numpy as np
import sympy as sym
import pydae.build_cffi as db

In [6]:
tau_d = 0.01
L_s = 0.01
R_s = 0.01
K_p = L_s/tau_d
K_i = R_s/tau_d
params_dict = {'L_s':L_s,'R_s':R_s,
               'N_pp':2.0,'Phi':1.0,
              'K_p':K_p,'K_i':K_i,
              'R_w':0.316,'G':9.81,'M':1200.0,'K_w':1.0,'C_rr':0.03,'Rho':1.225,'S_f':2.13,'C_x':0.32,'K_sign':100}


u_ini_dict = {'tau_r_ref':0.0,'i_d_ref':0.0,'beta':0.0}  # for the initialization problem
u_run_dict = {'tau_r_ref':0.0,'i_d_ref':0.0,'beta':0.0}  # for the running problem (here initialization and running problem are the same)


x_list = ['i_d','i_q','xi_i_d','xi_i_q','nu','x_pos']    # [inductor current, PI integrator]
y_ini_list = ['tau_r','omega_e','v_d','v_q','i_q_ref','snu','omega_r'] # for the initialization problem
y_run_list = ['tau_r','omega_e','v_d','v_q','i_q_ref','snu','omega_r'] # for the running problem (here initialization and running problem are the same)

sys_vars = {'params':params_dict,
            'u_list':u_run_dict,
            'x_list':x_list,
            'y_list':y_run_list}

exec(db.sym_gen_str())  # exec to generate the required symbolic varables and constants

In [7]:
# pmsm:
di_d = 1/L_s * (  v_d + L_s * omega_e * i_q - R_s * i_d  )
di_q = 1/L_s * (  v_q - L_s * omega_e * i_d - omega_e * Phi - R_s * i_q  )
eq_tau_r = -tau_r + 3/2*Phi*N_pp*i_q
eq_omega_e = -omega_e + N_pp*omega_r

# ctrl:
epsilon_i_d =  i_d_ref - i_d
epsilon_i_q =  i_q_ref - i_q
u_d = K_p*epsilon_i_d + K_i*xi_i_d
u_q = K_p*epsilon_i_q + K_i*xi_i_q

eq_v_d = -u_d + v_d + L_s * omega_e * i_q 
eq_v_q = -u_q + v_q - L_s * omega_e * i_d -  omega_e * Phi 
dxi_i_d = epsilon_i_d
dxi_i_q = epsilon_i_q
eq_i_q_ref = - tau_r_ref + 3/2*Phi*N_pp*i_q_ref

# mech:
tau_w = tau_r/K_w
f_w = tau_w/R_w
f_b = G*M*sym.sin(beta)
f_d = 0.5*Rho*S_f*C_x*nu**2*snu
f_r = C_rr*G*M*snu
p_r = tau_r*omega_r

dnu = 1/M*(f_w - f_d - f_r - f_b)
dx_pos = nu - x_pos*1e-4

eq_snu = -snu + (1/(1+sym.exp(-nu*K_sign)))*2-1
eq_omega_r =-omega_r +  nu/(K_w*R_w)

In [8]:
f_list = [di_d,di_q,dxi_i_d,dxi_i_q,dnu,dx_pos]
g_list = [eq_tau_r,eq_omega_e,eq_v_d,eq_v_q,eq_i_q_ref,eq_snu,eq_omega_r]
h_dict = {'u_d':u_d,'u_q':u_q,'tau_r_ref':tau_r_ref,
          'v_pk':(v_d**2+v_q**2)**0.5,
          'i_pk':(i_d**2+i_q**2)**0.5,'omega_r':omega_r,
         'f_w':f_w,'f_d':f_d,'f_r':f_r,'tau_r':tau_r,'p_r':p_r,'nu_kmh':3.6*nu}

sys = {'name':f"pmsm_mech_ctrl",
       'params_dict':params_dict,
       'f_list':f_list,
       'g_list':g_list,
       'x_list':x_list,
       'y_ini_list':y_list,
       'y_run_list':y_list,
       'u_run_dict':u_run_dict,
       'u_ini_dict':u_ini_dict,
       'h_dict':h_dict
      }

sys = db.build(sys)


C:\Users\jmmau\anaconda3\lib\site-packages\sympy\matrices\repmatrix.py:98: SymPyDeprecationWarning: 

non-Expr objects in a Matrix has been deprecated since SymPy 1.9. Use
list of lists, TableForm or some other data structure instead. See
https://github.com/sympy/sympy/issues/21497 for more info.

  SymPyDeprecationWarning(


Compilation time: 7.89 s
